<a href="https://colab.research.google.com/github/676647/ML-aaignment-2/blob/main/RuccoBeatsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio pandas numpy scikit-learn joblib librosa pydub

In [2]:
import gradio as gr
import pandas as pd
import numpy as np
import joblib
import librosa
import os

In [15]:
try:
    final_rf = joblib.load('final_rf.pkl')
    scaler = joblib.load('scaler.pkl')
    le = joblib.load('label_encoder.pkl')
except FileNotFoundError:
    print("Error: Model files not found. Please upload 'final_rf.pkl', 'scaler.pkl', and 'label_encoder.pkl' to the Colab environment.")
    exit()

FEATURE_NAMES = [
    'duration_ms', 'danceability', 'energy', 'loudness', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'
]

print("Setup complete. Model components loaded successfully.")

Setup complete. Model components loaded successfully.


In [26]:
!pip install mutagen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 4.2 MB/s eta 0:00:00


In [27]:
import requests
import json
import os
import joblib
from mutagen.mp3 import MP3

API_URL = "https://api.reccobeats.com/v1/analysis/audio-features"

In [28]:
MODEL_FEATURE_NAMES = [
    'duration_ms', 'danceability', 'energy', 'loudness', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'
]

API_FEATURE_KEYS = [
    'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo'
]


def extract_features_from_mp3_api(audio_filepath):

    if not os.path.exists(audio_filepath):
        raise FileNotFoundError(f"Audio file not found: {audio_filepath}")

    try:
        audio = MP3(audio_filepath)

        duration_ms = audio.info.length * 1000
    except Exception as e:

        print(f"Error calculating local duration: {e}")
        raise ValueError("Could not read local audio file duration.")


    url = "https://api.reccobeats.com/v1/analysis/audio-features"
    headers = {'Accept': 'application/json'}

    with open(audio_filepath, 'rb') as f:
        files = {'audioFile': (os.path.basename(audio_filepath), f, 'application/octet-stream')}
        response = requests.post(url, headers=headers, files=files)

    response.raise_for_status()

    data = response.json()

    extracted_values = [duration_ms]

    for key in API_FEATURE_KEYS:
        if key not in data:
            raise KeyError(f"API response missing expected key: '{key}'. Response keys: {list(data.keys())}")

        extracted_values.append(data[key])

    return extracted_values

In [29]:
def predict_genre_from_file (audio_file_path):
  if audio_file_path is None:
    return "### ⚠️ Please upload an audio file first."

  try:
      raw_features = extract_features_from_mp3_api(audio_file_path)
  except Exception as e:
      return f"### ❌ Error! Could not process audio file.\nDetails: {e}"

  input_df = pd.DataFrame([raw_features], columns=FEATURE_NAMES)
  input_scaled = scaler.transform(input_df)
  proba_array = final_rf.predict_proba(input_scaled)[0]
  genre_labels = le.classes_

  results = pd.DataFrame ({'Genre': genre_labels, 'Probability': proba_array})
  results = results.sort_values(by='Probability', ascending=False)
  results['Probability (%)'] = (results['Probability']*100).round(2).astype(str) + '%'

  output_text = "### 🎤 Predicted Genre:\n"
  output_text += f"**{results.iloc[0]['Genre']}** ({results.iloc[0]['Probability (%)']})\n\n"
  output_text += "### 🎼 All Genre Probabilities:\n"
  output_text += results[['Genre', 'Probability (%)']].to_markdown(index=False)

  return output_text

audio_input = gr.Audio(
    type="filepath",
    label="Upload MP3 or WAV (Max 5 MB)",
    sources=["upload"]
)

iface = gr.Interface(
    fn=predict_genre_from_file,
    inputs=audio_input,
    outputs="markdown",
    title="🎶 Spotify Music Genre Detector via File Upload",
    description="Upload an audio file (MP3/WAV) under 5MB to extract acoustic features and receive a ranked genre prediction.",
)

if __name__ == "__main__":
    iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29c89a20557fee5990.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
